In [18]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [21]:
dados_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco","idade_do_modelo","km_por_ano"]]
y_azar = dados_azar["vendido"]
dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [23]:
from sklearn.model_selection import cross_validate
from sklearn.dummy import DummyClassifier
import numpy as np

SEED = 301
np.random.seed(SEED)

modelo = DummyClassifier()
results = cross_validate(modelo, x_azar, y_azar, cv = 10, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com dummy stratified, 10 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao)*100))

Accuracy com dummy stratified, 10 = [58.00, 58.00]


In [24]:
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeClassifier

SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv =10, return_train_score = False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com dummy stratified, 10 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao)*100))

Accuracy com dummy stratified, 10 = [73.83, 77.73]


In [25]:
# gerando dados aleatorios de modelo de carro para simulado de agrupamento ao usar nosso estimador

np.random.seed(SEED)
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)
dados.modelo - dados.modelo + abs(dados.modelo.min()) + 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,16
1,40557.96,1,20,12622.05362,22
2,89627.50,0,12,11440.79806,12
3,95276.14,0,3,43167.32682,4
4,117384.68,1,4,12770.11290,3


In [26]:
def imprime_resultados(results):
    media = results['test_score'].mean() * 100
    desvio = results['test_score'].std() * 100
    print("Accuracy médio: %.2f" % media)
    print("Intervalo [%.2f, %.2f]" %( media - 2 * desvio, media + 2 * desvio ))

In [29]:
# GroupKFols em um pipeline com StandardScaler e SVC
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler 
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador',modelo)])

cv = GroupKFold(n_splits = 10)
results = cross_validate(pipeline, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 76.68
Intervalo [74.28, 79.08]


In [31]:
# GroupKFold para analisar como o modelo se comporta com novos grupos
# Na validação cruzada treinamos para saber quão bom seria nosso modelo no mundo real
from sklearn.model_selection import GroupKFold
from sklearn.tree import DecisionTreeClassifier

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score = False)
imprime_resultados(results) 

Accuracy médio: 75.78
Intervalo [73.67, 77.90]


sklearn.tree._classes.DecisionTreeClassifier

# Testando parâmetros

In [8]:
def roda_arvore_decisao(max_depth):
    SEED = 301
    np.random.seed(SEED)

    cv = GroupKFold(n_splits = 10)
    modelos = DecisionTreeClassifier(max_depth=max_depth)
    results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score = True)
    train_score = results['train_score'].mean() * 100
    test_score = results['test_score'].mean() * 100
    print("Arvore max_depth = %d, treino = %.2f, test = %.2f" % (max_depth, train_score, test_score ))
    tabela = [max_depth, train_score, test_score]
    return tabela

resultados = [roda_arvore_decisao(i) for i in range(1, 33)]#33 exclusive 32 inclusivo  
resultados = pd.DataFrame(resultados, columns = ["max_depth", "train","test"])
resultados.head()

Arvore max_depth = 1, treino = 81.16, test = 77.19
Arvore max_depth = 2, treino = 81.16, test = 77.19
Arvore max_depth = 3, treino = 81.16, test = 77.19
Arvore max_depth = 4, treino = 81.16, test = 77.19
Arvore max_depth = 5, treino = 81.16, test = 77.19
Arvore max_depth = 6, treino = 81.16, test = 77.19
Arvore max_depth = 7, treino = 81.16, test = 77.19
Arvore max_depth = 8, treino = 81.16, test = 77.19
Arvore max_depth = 9, treino = 81.16, test = 77.19
Arvore max_depth = 10, treino = 81.16, test = 77.19
Arvore max_depth = 11, treino = 81.16, test = 77.19
Arvore max_depth = 12, treino = 81.16, test = 77.19
Arvore max_depth = 13, treino = 81.16, test = 77.19
Arvore max_depth = 14, treino = 81.16, test = 77.19
Arvore max_depth = 15, treino = 81.16, test = 77.19
Arvore max_depth = 16, treino = 81.16, test = 77.19
Arvore max_depth = 17, treino = 81.16, test = 77.19
Arvore max_depth = 18, treino = 81.16, test = 77.19
Arvore max_depth = 19, treino = 81.16, test = 77.19
Arvore max_depth = 20

,max_depth,train,test
0,1,81.158994,77.194204
1,2,81.158994,77.194204
2,3,81.158994,77.194204
3,4,81.158994,77.194204
4,5,81.158994,77.194204


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.lineplot(x = "max_depth", y = "train", data = resultados)
#sns.lineplot(x = "max_depth", y = "test", data = resultados)
#plt.legend(["Treino","Teste"])


<AxesSubplot:xlabel='max_depth', ylabel='train'>

In [ ]:
resultados.sort_values("test", ascending=False).head()

# Explorando hiper parâmetros em 2 dimensões

In [ ]:
def roda_arvore_decisao(max_depth, min_samples_leaf):
    SEED = 301
    np.random.seed(SEED)

    cv = GroupKFold(n_splits = 10)
    modelos - DecisonTreeClassifier(max_depth=max_depth, min_samples_leaf = min_samples_leaf)
    results = cross_alidate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score = False)
    train_score = results['train_score'].mean() * 100
    test_score = results['test_score'].mean() * 100
    print("Arvore max_depth = %d, min_samples_leaf = %.d, treino = %.2f, test = %.2f" % (max_depht, min_samples_leaf, train_score, test_score ))
    tabela = [max_depth, min_samples_leaf train_score, test_score]
    return tabela

def busca():
    resultados = []
    for max_depth in range(1, 33):
        for min_samples_leaf in [32, 64, 128, 256]:
            tabela = roda_arvore_decisão(max_depth, min_samples_leaf)
            resultados.append(tabela)
    resultados = pd.DataFrame(resultados, columns = ["max_depth", "min_samples_leaf", "train","test"])
    return resultados
 
resultados = busca()
resultados.head()

In [ ]:
resultados.sort_values("test", ascending=False).head()

In [ ]:
resultados.corr()

In [ ]:
sns.heatmap(corr)

In [ ]:
pd.scatter_matrix(resultados, figsize=(14, 8))

In [ ]:

def busca():
    resultados = []
    for max_depth in range(1, 33):
        for min_samples_leaf in [128, 192, 256, 512]:
            tabela = roda_arvore_decisão(max_depth, min_samples_leaf)
            resultados.append(tabela)
    resultados = pd.DataFrame(resultados, columns = ["max_depth", "min_samples_leaf", "train","test"])
    return resultados

resultados = busca()
resultados.head()